In [27]:
from ollama import Client

# Initialize the OLlama client
OLLAMA_HOST = "http://127.0.0.1:11434"
ollama_client = Client(host=OLLAMA_HOST)

# Load the few-shot examples
with open("prompts/open_ended_ClinicR.txt", 'r') as f:
    few_shot_cot = f.read()

# Define constants
reasoning_delimiter = "Answer: "
output_delimiter = "Q:"
instruction = (
    "Use just the given patient history to answer the question. "
    "Do not assume any further information about the patient. "
    "Strictly Limit your response to 200 words."
)


def ask_ollama(ollama_client, question, few_shot_cot, instruction, model="llama3.2:1b"):
    """
    Query the OLlama client with the input prompt and process the response.
    
    Args:
        ollama_client (Client): Initialized OLlama client.
        question (str): The question to ask.
        few_shot_cot (str): Few-shot examples for context.
        instruction (str): Instruction for the LLM.
        model (str): The model name to use.
    
    Returns:
        dict: The LLM's reasoning and answer.
    """
    # Prepare the input string
    instring = f"{few_shot_cot}\n\n{instruction}\nQ: {question}\nA: Let's think step-by-step."
    print("DEBUG: Input to OLlama:", instring)  # Debugging line
    # Query the OLlama model
    try:
        # Generate the response
        response = ollama_client.generate(model=model, prompt=instring)
        print("DEBUG: Full Response from OLlama:", response)  # Debugging line
        
        # Extract the full output from the response
        full_output = response.response.strip()  # Extract text from 'response' key
        
        if reasoning_delimiter in full_output:
            reasoning, answer = full_output.split(reasoning_delimiter, 1)
            return {"reasoning": reasoning.strip(), "answer": answer.strip()}
        else:
            return {"reasoning": full_output, "answer": "<parsing error>"}
    except Exception as e:
        print(f"Error querying OLlama: {e}")
        return {"reasoning": "<error>", "answer": "<error>"}



# Jupyter Notebook Functionality
def run_ollama_in_notebook(question=None, model="llama3.2:1b"):
    """
    Run OLlama client in a Jupyter Notebook environment.
    
    Args:
        question (str): The medical question to answer. If None, prompt user for input.
        model (str): The model name to use.
    
    Returns:
        None: Prints the reasoning and answer.
    """
    # Use the provided question or ask for input
    if not question:
        question = input("Enter your question: ")

    # Call the OLlama client
    output = ask_ollama(ollama_client, question, few_shot_cot, instruction, model=model)
    
    # Display the output
    print("\n=== Output ===")
    print(f"The output is: {output['answer']}")
    print(f"The Reasoning is as follows: {output['reasoning']}")


In [28]:
run_ollama_in_notebook("A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Describe the appropriate ethical action that the resident must take in this condition")


DEBUG: Input to OLlama: Use just the given patient history to answer the question. Do not assume any further information about the patient. Strictly Limit your response to 200 words.
Q: A 4670-g (10-lb 5-oz) male newborn is delivered at term to a 26-year-old woman after prolonged labor. Apgar scores are 9 and 9 at 1 and 5 minutes. Examination in the delivery room shows swelling, tenderness, and crepitus over the left clavicle. There is decreased movement of the left upper extremity. Movement of the hands and wrists are normal. A grasping reflex is normal in both hands. An asymmetric Moro reflex is present. The remainder of the examination shows no abnormalities and an anteroposterior x-ray confirms the diagnosis. What is the most suitable course of action to manage this case?
A: Let's think step-by-step. Prolonged labour would mean it would be a difficult child birth. Apgar score is 9 and does not drop after repeated examination which means that there is no respiratory or neurological 